In [1]:
import numpy as np
from tensorflow import keras

In [2]:
def generate_time_series(batch_size, n_steps):
    freq1 ,freq2, offset1, offset2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offset1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offset2) * (freq2 * 20 + 20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

In [3]:
n_steps = 50
series = generate_time_series(10000, n_steps + 10)

train_X = series[:7500, :n_steps]
train_y = series[:7500, -10:]
valid_X = series[7500:, :n_steps]
valid_y = series[7500:, -10:]

In [4]:
valid_pred_naive_y = np.empty((2500, 10, 1))
for i, value in enumerate(valid_X[:, -1]):
    valid_pred_naive_y[i] = [[value]] * 10
    
np.mean(keras.losses.mean_squared_error(valid_y, valid_pred_naive_y))

0.26499531246006797

In [5]:
early_stop_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [6]:
model_linear = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(10)
])

model_linear.compile(loss='mean_squared_error', optimizer='Adam')
model_linear.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 0s 988us/step - loss: 0.1222 - val_loss: 0.0620
Epoch 2/50
235/235 [==============================] - 0s 662us/step - loss: 0.0520 - val_loss: 0.0429
Epoch 3/50
235/235 [==============================] - 0s 669us/step - loss: 0.0395 - val_loss: 0.0352
Epoch 4/50
235/235 [==============================] - 0s 675us/step - loss: 0.0335 - val_loss: 0.0310
Epoch 5/50
235/235 [==============================] - 0s 676us/step - loss: 0.0298 - val_loss: 0.0282
Epoch 6/50
235/235 [==============================] - 0s 664us/step - loss: 0.0272 - val_loss: 0.0262
Epoch 7/50
235/235 [==============================] - 0s 682us/step - loss: 0.0255 - val_loss: 0.0248
Epoch 8/50
235/235 [==============================] - 0s 671us/step - loss: 0.0242 - val_loss: 0.0236
Epoch 9/50
235/235 [==============================] - 0s 659us/step - loss: 0.0231 - val_loss: 0.0230
Epoch 10/50
235/235 [==============================] - 0s 664us/step - loss: 0.022

In [7]:
model_baseline = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(10)
])

model_baseline.compile(loss='mean_squared_error', optimizer='Adam')
model_baseline.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 0s 993us/step - loss: 0.0803 - val_loss: 0.0419
Epoch 2/50
235/235 [==============================] - 0s 757us/step - loss: 0.0331 - val_loss: 0.0273
Epoch 3/50
235/235 [==============================] - 0s 753us/step - loss: 0.0241 - val_loss: 0.0218
Epoch 4/50
235/235 [==============================] - 0s 731us/step - loss: 0.0197 - val_loss: 0.0183
Epoch 5/50
235/235 [==============================] - 0s 790us/step - loss: 0.0171 - val_loss: 0.0162
Epoch 6/50
235/235 [==============================] - 0s 738us/step - loss: 0.0155 - val_loss: 0.0152
Epoch 7/50
235/235 [==============================] - 0s 750us/step - loss: 0.0142 - val_loss: 0.0139
Epoch 8/50
235/235 [==============================] - 0s 756us/step - loss: 0.0132 - val_loss: 0.0133
Epoch 9/50
235/235 [==============================] - 0s 731us/step - loss: 0.0124 - val_loss: 0.0123
Epoch 10/50
235/235 [==============================] - 0s 740us/step - loss: 0.011

In [8]:
model_simpleRNN = keras.models.Sequential([
    keras.layers.SimpleRNN(10, input_shape=[None, 1])
])

model_simpleRNN.compile(loss='mean_squared_error', optimizer='Adam')
model_simpleRNN.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 2s 5ms/step - loss: 0.1452 - val_loss: 0.1066
Epoch 2/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0922 - val_loss: 0.0783
Epoch 3/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0707 - val_loss: 0.0633
Epoch 4/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0586 - val_loss: 0.0548
Epoch 5/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0518 - val_loss: 0.0496
Epoch 6/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0477 - val_loss: 0.0456
Epoch 7/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0447 - val_loss: 0.0430
Epoch 8/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0432 - val_loss: 0.0414
Epoch 9/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0418 - val_loss: 0.0413
Epoch 10/50
235/235 [==============================] - 1s 5ms/step - loss: 0.0413 - val_loss: 0.0403

In [9]:
model_deepRNN = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None,1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])

model_deepRNN.compile(loss='mean_squared_error', optimizer='Adam')
model_deepRNN.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 4s 12ms/step - loss: 0.0710 - val_loss: 0.0300
Epoch 2/50
235/235 [==============================] - 3s 11ms/step - loss: 0.0231 - val_loss: 0.0179
Epoch 3/50
235/235 [==============================] - 3s 11ms/step - loss: 0.0162 - val_loss: 0.0139
Epoch 4/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0139 - val_loss: 0.0167
Epoch 5/50
235/235 [==============================] - 3s 11ms/step - loss: 0.0130 - val_loss: 0.0111
Epoch 6/50
235/235 [==============================] - 3s 11ms/step - loss: 0.0114 - val_loss: 0.0105
Epoch 7/50
235/235 [==============================] - 3s 12ms/step - loss: 0.0106 - val_loss: 0.0105
Epoch 8/50
235/235 [==============================] - 3s 11ms/step - loss: 0.0104 - val_loss: 0.0120
Epoch 9/50
235/235 [==============================] - 3s 11ms/step - loss: 0.0104 - val_loss: 0.0100
Epoch 10/50
235/235 [==============================] - 3s 11ms/step - loss: 0.0100 - val_lo

In [10]:
model_singleLSTM = keras.models.Sequential([
    keras.layers.LSTM(10, input_shape=[None,1]),
])

model_singleLSTM.compile(loss='mean_squared_error', optimizer='Adam')
model_singleLSTM.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 4s 11ms/step - loss: 0.1079 - val_loss: 0.0645
Epoch 2/50
235/235 [==============================] - 2s 10ms/step - loss: 0.0504 - val_loss: 0.0426
Epoch 3/50
235/235 [==============================] - 2s 10ms/step - loss: 0.0401 - val_loss: 0.0390
Epoch 4/50
235/235 [==============================] - 2s 10ms/step - loss: 0.0367 - val_loss: 0.0355
Epoch 5/50
235/235 [==============================] - 2s 10ms/step - loss: 0.0349 - val_loss: 0.0338
Epoch 6/50
235/235 [==============================] - 2s 10ms/step - loss: 0.0335 - val_loss: 0.0334
Epoch 7/50
235/235 [==============================] - 2s 10ms/step - loss: 0.0324 - val_loss: 0.0316
Epoch 8/50
235/235 [==============================] - 2s 10ms/step - loss: 0.0314 - val_loss: 0.0310
Epoch 9/50
235/235 [==============================] - 2s 10ms/step - loss: 0.0307 - val_loss: 0.0302
Epoch 10/50
235/235 [==============================] - 2s 10ms/step - loss: 0.0299 - val_lo

In [11]:
model_deepLSTM = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None,1]),
    keras.layers.LSTM(20),
    keras.layers.Dense(10)
])

model_deepLSTM.compile(loss='mean_squared_error', optimizer='Adam')
model_deepLSTM.fit(train_X, train_y, epochs=50, validation_data=(valid_X, valid_y), callbacks=[early_stop_cb])

Epoch 1/50
235/235 [==============================] - 7s 23ms/step - loss: 0.0683 - val_loss: 0.0328
Epoch 2/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0279 - val_loss: 0.0242
Epoch 3/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0235 - val_loss: 0.0224
Epoch 4/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0216 - val_loss: 0.0213
Epoch 5/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0198 - val_loss: 0.0192
Epoch 6/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0183 - val_loss: 0.0193
Epoch 7/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0166 - val_loss: 0.0154
Epoch 8/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0159 - val_loss: 0.0171
Epoch 9/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0152 - val_loss: 0.0159
Epoch 10/50
235/235 [==============================] - 5s 21ms/step - loss: 0.0148 - val_lo